In [ ]:
TP-07
1- bibliotheque Kafka-python
                                  

In [ ]:
pip install kafka-python

In [ ]:
2-Creation d'un fichier iot_producer.py:
pour faire une simulation de capteur qui envoie des donnees vers un topic Kafka

In [ ]:
import json
import time
import random
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

machines = ["M001", "M002", "M003"]

print("Kafka IoT Producer started ...")

try:
    while True:
        data = {
            "machine_id": random.choice(machines),
            "temperature": round(random.uniform(20, 100), 2),
            "vibration": round(random.uniform(0, 5), 3),
            "timestamp": time.time()
        }

        producer.send("tp07-iot", value=data)
        print("Sent:", data)
        time.sleep(1)

except KeyboardInterrupt:
    print("Producer stopped")

producer.close()


In [ ]:
3-Creation d'un fichier iot_iot_consumer:
Pour receptionner  les donnees envoyees par kafka   et stocker dans CSV et fait des alertes

In [ ]:
import json
import csv
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    "tp07-iot",
    bootstrap_servers="localhost:9092",
    auto_offset_reset="earliest",
    group_id="iot-group",
    value_deserializer=lambda x: json.loads(x.decode("utf-8"))
)

print("Kafka IoT Consumer started ...")

with open("iot_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["machine_id", "temperature", "vibration", "timestamp"])

    for msg in consumer:
        data = msg.value
        print("Received:", data)

        writer.writerow([
            data["machine_id"],
            data["temperature"],
            data["vibration"],
            data["timestamp"]
        ])

        if data["temperature"] > 80:
            print("⚠️ ALERT: High Temperature for", data["machine_id"])


In [ ]:
4-Execution de KAFKA

In [ ]:
 C:\Users\user\Desktop\TP07_KAFKA> bin\windows\zookeeper-server-start.bat config\zookeeper.properties
>> # Kafka Broker


In [ ]:
5- excution de fichier iot_consumer.py

In [ ]:
python iot_consumer.py


In [ ]:
6- excution de fichier iot_produce.py

In [ ]:
python iot_produce.py


In [ ]:
resultat:Sent: {'machine_id': 'M002', 'temperature': 45.67, 'vibration': 2.345, 'timestamp': 1700000000.123456}
Received: {'machine_id': 'M002', 'temperature': 45.67, 'vibration': 2.345, 'timestamp': 1700000000.123456}
Sent: {'machine_id': 'M001', 'temperature': 85.23, 'vibration': 3.456, 'timestamp': 1700000001.123456}
⚠️ ALERT: High Temperature for M001         